In [1]:
#Import neccessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' #path to the tesseract file
#Install pytesseract if its not done before 
import os
import pandas as pd
from time import process_time

In [4]:
def image_to_gray(original_file_path):
    read_image=cv2.imread(original_file_path, 0)
    crop_img1 = read_image[285:1000, 635:1025]
    cv2.imwrite(crop1, crop_img1)
    crop_img2 = read_image[285:1000, 1575:1850]
    cv2.imwrite(crop2, crop_img2)
    crop_img=cv2.hconcat([crop_img1, crop_img2])
    cv2.imwrite(crop_final, crop_img)
    read_crop_image=cv2.imread(crop_final,0)
    convert_bin,grey_scale = cv2.threshold(read_crop_image,0,255,cv2.THRESH_TOZERO | cv2.THRESH_OTSU)
    grey_scale=255-grey_scale
    h, w = grey_scale.shape[:2]
    grey_scale = cv2.resize(grey_scale, (int(w*1.75), int(h*1.75)))
    plt.imsave(gray_img, grey_scale, cmap='gray')

def create_col_gridlines(gray_image_path):
    # read image
    img = cv2.imread(gray_image_path)
    hh, ww = img.shape[:2]
    # convert to grayscale 
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # threshold gray image
    thresh = cv2.threshold(gray, 254, 255, cv2.THRESH_BINARY)[1]
    # count number of non-zero pixels in each column
    count = np.count_nonzero(thresh, axis=0)
    # threshold count at hh (height of image)
    count_thresh = count.copy()
    count_thresh[count==hh] = 255
    count_thresh[count<hh] = 0
    count_thresh = count_thresh.astype(np.uint8)
    # get contours
    contours = cv2.findContours(count_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    # loop over contours and get bounding boxes and xcenter and draw vertical line at ycenter
    result = img.copy()
    for cntr in contours:
        # must transpose x,y and w,h since count is one-dimensional but represents each column
        y,x,h,w = cv2.boundingRect(cntr)
        xcenter = x+w//2
        cv2.line(result, (xcenter,0), (xcenter, hh-1), (0, 0, 0), 2)
    # save results
    cv2.imwrite(col_line, result)

def create_row_gridlines(col_file_path,image_index):
    img = cv2.imread(col_file_path,0)
    hh, ww = img.shape[:2]
    # average gray image to one column
    column = cv2.resize(img, (1,hh), interpolation = cv2.INTER_AREA)
    # threshold on white
    thresh = cv2.threshold(column, 250, 255, cv2.THRESH_BINARY)[1]
    # get contours
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    # loop over contours and get bounding boxes and ycenter and draw horizontal line at ycenter
    result = img.copy()
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
        ycenter = y+h//2
        cv2.line(result, (0,ycenter), (ww-1,ycenter), (0, 0, 0), 2)
    # write results
    x=row_col_path+'\\'+f'img{image_index}.jpg'
    cv2.imwrite(x, result)

def c_data(file):
    img = cv2.imread(file,0)
    #Inverted Image with global thresh holding
    thresh,img_bin = cv2.threshold(img,128,255,cv2.THRESH_BINARY)
    img_bin = 255-img_bin #variable used at horizontal kernel
    thresh1,img_bin_otsu = cv2.threshold(img_bin,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)#used at ver kernel
    #define kernel
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    #Create vertical lines
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, np.array(img).shape[1]//50))
    eroded_image = cv2.erode(img_bin_otsu, vertical_kernel, iterations=5)
    vertical_lines = cv2.dilate(eroded_image, vertical_kernel, iterations=5)
    #create horizontal lines
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (np.array(img).shape[1]//80, 1))
    horizontal_lines = cv2.erode(img_bin, hor_kernel, iterations=5)
    horizontal_lines = cv2.dilate(horizontal_lines, hor_kernel, iterations=5)
    #create vertical horizontal lines
    vertical_horizontal_lines = cv2.addWeighted(vertical_lines, 0.25, horizontal_lines, 0.25, 0.0)
    vertical_horizontal_lines = cv2.erode(~vertical_horizontal_lines, kernel, iterations=3)
    thresh, vertical_horizontal_lines = cv2.threshold(vertical_horizontal_lines,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    bitxor = cv2.bitwise_xor(img,vertical_horizontal_lines)
    bitnot = cv2.bitwise_not(bitxor)
    #create contours
    contours, hierarchy = cv2.findContours(vertical_horizontal_lines, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    boundingBoxes = [cv2.boundingRect(contour) for contour in contours]
    (contours, boundingBoxes) = zip(*sorted(zip(contours, boundingBoxes),key=lambda x:x[1][1]))
    boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if (w<1000 and h<500 and w>180 ):
            image = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            boxes.append([x,y,w,h])
    #Create rows
    rows=[]
    columns=[]
    heights = [boundingBoxes[i][3] for i in range(len(boundingBoxes))]
    mean = np.mean(heights)
    #print("Mean: ",mean)
    columns.append(boxes[0])
    previous=boxes[0]
    for i in range(1,len(boxes)):
        if(boxes[i][1]<=previous[1]+mean/2):
            columns.append(boxes[i])
            previous=boxes[i]
            if(i==len(boxes)-1):
                rows.append(columns)
        else:
            rows.append(columns)
            columns=[]
            previous = boxes[i]
            columns.append(boxes[i])
    #print("Rows")
    #print(rows)
    #for row in rows:
        #x='dummy'
        #print(row)
    for i in range(0,len(rows)):
        for j in range (0,len(rows[i])):
            try:
                rows[i].remove([])
            except:
                x='Empty strings are deleted'
    total_cells=0
    for i in range(0,len(rows)):
        #print(f"Value of row {i}")
        #print(rows[i])
        #print(f"Length of row {i} is : {len(rows[i])}")
        if len(rows[i]) > total_cells:
            total_cells = len(rows[i])
    #print("Total Cells: ",total_cells)
    center = [int(rows[i][j][0]+rows[i][j][2]/2) for j in range(len(rows[i])) if rows[0]]
    center=np.array(center)
    center.sort()
    #print("Center: ",center)
    boxes_list = []
    #print(len(rows))
    for i in range(len(rows)):
        l=[]
        for k in range(total_cells):
            l.append([])
        for j in range(len(rows[i])):
            diff = abs(center-(rows[i][j][0]+rows[i][j][2]/4))
            minimum = min(diff)
            indexing = list(diff).index(minimum)
            l[indexing].append(rows[i][j])
        boxes_list.append(l)
    #print("Length of the rows: ",len(boxes_list))
    #boxes_list.sort(reverse=True)
    #print(boxes_list)
    for box in boxes_list:
        box.sort(key=[].__eq__)
        #print(box)
    #create dataframe
    dataframe_final=[]
    for i in range(len(boxes_list)):
        for j in range(len(boxes_list[i])):
            s=''
            if(len(boxes_list[i][j])==0):
                dataframe_final.append(' ')
            else:
                for k in range(len(boxes_list[i][j])):
                    y,x,w,h = boxes_list[i][j][k][0],boxes_list[i][j][k][1], boxes_list[i][j][k][2],boxes_list[i][j][k][3]
                    roi = bitnot[x:x+h, y:y+w]
                    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 1))
                    border = cv2.copyMakeBorder(roi,2,2,2,2, cv2.BORDER_CONSTANT,value=[255,255])
                    resizing = cv2.resize(border, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
                    dilation = cv2.dilate(resizing, kernel,iterations=1)
                    erosion = cv2.erode(dilation, kernel,iterations=2)                
                    out = pytesseract.image_to_string(erosion)
                    if(len(out)==0):
                        out = pytesseract.image_to_string(erosion)
                    s = s +" "+ out
                dataframe_final.append(s)
    arr = np.array(dataframe_final)
    dataframe = pd.DataFrame(arr.reshape(len(rows), total_cells))
    data = dataframe.style.set_properties(align="left")
    #print(dataframe)
    return dataframe

In [2]:
my_path = 'G:\Doc\Rok\Individual Power Ranking'
crop1='G:\Doc\Rok\Crop1.jpg'
crop2='G:\Doc\Rok\Crop2.jpg'
crop_final='G:\Doc\Rok\Cropf.jpg'
gray_img='G:\Doc\Rok\gray.jpg'
col_line='G:\Doc\Rok\col.jpg'
row_col_path='G:\Doc\Rok' #just path dont add jpg file

In [3]:
#Create list of all files in the path
files = os.listdir(my_path)
image_list=[]
for f in files:
    x=my_path+'\\'+f
    image_list.append(x)
print(image_list)

['G:\\Doc\\Rok\\Individual Power Ranking\\Screenshot_2022-05-02-10-23-14-582_com.lilithgame.roc.gp.jpg', 'G:\\Doc\\Rok\\Individual Power Ranking\\Screenshot_2022-05-02-10-23-22-451_com.lilithgame.roc.gp.jpg', 'G:\\Doc\\Rok\\Individual Power Ranking\\Screenshot_2022-05-02-10-23-29-481_com.lilithgame.roc.gp.jpg', 'G:\\Doc\\Rok\\Individual Power Ranking\\Screenshot_2022-05-02-10-23-35-900_com.lilithgame.roc.gp.jpg', 'G:\\Doc\\Rok\\Individual Power Ranking\\Screenshot_2022-05-02-10-23-41-220_com.lilithgame.roc.gp.jpg', 'G:\\Doc\\Rok\\Individual Power Ranking\\Screenshot_2022-05-02-10-23-48-357_com.lilithgame.roc.gp.jpg', 'G:\\Doc\\Rok\\Individual Power Ranking\\Screenshot_2022-05-02-10-23-55-729_com.lilithgame.roc.gp.jpg', 'G:\\Doc\\Rok\\Individual Power Ranking\\Screenshot_2022-05-02-10-24-00-847_com.lilithgame.roc.gp.jpg', 'G:\\Doc\\Rok\\Individual Power Ranking\\Screenshot_2022-05-02-10-24-08-175_com.lilithgame.roc.gp.jpg', 'G:\\Doc\\Rok\\Individual Power Ranking\\Screenshot_2022-05-02-

In [8]:
#creating row and column gridlines
for i in range(0,len(image_list)):
    image_to_gray(image_list[i])
    create_col_gridlines(gray_img)
    create_row_gridlines(col_line,i)

In [21]:
final_img_path='G:\Doc\Rok'
#Create list of all files in the path
files = os.listdir(final_img_path)
final_image_list=[]
for f in files:
    if f.startswith('img'):
        final_image_list.append(f)

print(final_image_list)
#print(len(image_list)) #strings cant be sorted by number

['img0.jpg', 'img1.jpg', 'img10.jpg', 'img11.jpg', 'img12.jpg', 'img13.jpg', 'img14.jpg', 'img15.jpg', 'img16.jpg', 'img17.jpg', 'img18.jpg', 'img19.jpg', 'img2.jpg', 'img20.jpg', 'img21.jpg', 'img22.jpg', 'img23.jpg', 'img24.jpg', 'img25.jpg', 'img26.jpg', 'img27.jpg', 'img28.jpg', 'img29.jpg', 'img3.jpg', 'img30.jpg', 'img31.jpg', 'img32.jpg', 'img33.jpg', 'img34.jpg', 'img35.jpg', 'img36.jpg', 'img37.jpg', 'img38.jpg', 'img39.jpg', 'img4.jpg', 'img40.jpg', 'img41.jpg', 'img42.jpg', 'img43.jpg', 'img44.jpg', 'img45.jpg', 'img46.jpg', 'img47.jpg', 'img48.jpg', 'img49.jpg', 'img5.jpg', 'img50.jpg', 'img51.jpg', 'img52.jpg', 'img53.jpg', 'img54.jpg', 'img55.jpg', 'img56.jpg', 'img57.jpg', 'img58.jpg', 'img59.jpg', 'img6.jpg', 'img60.jpg', 'img61.jpg', 'img62.jpg', 'img63.jpg', 'img64.jpg', 'img65.jpg', 'img66.jpg', 'img7.jpg', 'img8.jpg', 'img9.jpg']


In [18]:
finaldata=[]
t1= process_time() 
for i in range(0,len(final_image_list)):
    t1_start = process_time() 
    file=final_img_path+"\\"+"img"+str(i)+".jpg"
    x=c_data(file)
    finaldata.append(x)
    t1_stop = process_time()
    print(f"Image {i+1} processed and Elapsed time is {(t1_stop-t1_start)*10} seconds")
t2=process_time()
print(f"Total time taken is {(t2-t1)/6} minutes")  

G:\Doc\Rok\img0.jpg
Image 1 processed and Elapsed time is 6.5625 seconds
G:\Doc\Rok\img1.jpg
Image 2 processed and Elapsed time is 4.84375 seconds
G:\Doc\Rok\img2.jpg
Image 3 processed and Elapsed time is 5.3125 seconds
G:\Doc\Rok\img3.jpg


KeyboardInterrupt: 

In [19]:
x=pd.DataFrame()
for i in range(0,len(finaldata)):
    x=pd.concat([x,finaldata[i]],ignore_index=True)

In [20]:
x

,0,1,2,3
0,[BU83] Prot3us Panda,"129,929,537",,
1,Bamboo Underworld,,,
2,[BU83] Vdawg,"114,053,695",,
3,Bamboo Underworld,,,
4,[BU83] Katana X%,"107,939,972",,
...,...,...,...,...
34,[BU83] Chief Panda,"85,230,765",NaN,NaN
35,Bamboo Underworld,,NaN,NaN
36,[BU83] Banshiii,"85,219,696",NaN,NaN
37,Bamboo Underworld,,NaN,NaN


In [13]:
file_name = 'Ind Pwr Ranking.xlsx'
x.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


#  All Done
    Data Cleaning in Excel using Filters and converting numbers from string to number and use conditional formatting to find transcription Error and correcting it.